In [2]:
!pip install fastapi
!pip install kubernetes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from fastapi import FastAPI, HTTPException
from kubernetes import client, config
from kubernetes.client.rest import ApiException
from kubernetes import client, config
from kubernetes.config import ConfigException
from fastapi import FastAPI, HTTPException, Path

In [4]:
pip install kubernetes

In [5]:
import os

In [6]:
# Specify the path to your kube config file
config_file_path = os.path.expanduser('~/.kube/config')
print(config_file_path)  # Print the path to verify

/root/.kube/config


In [7]:
!pip install prometheus_client
from prometheus_client import CollectorRegistry

In [8]:
# Prometheus configuration
PROMETHEUS_PUSHGATEWAY = "http://prometheus-pushgateway:9091"
registry = CollectorRegistry()

In [9]:
from fastapi import FastAPI  # Import FastAPI
app = FastAPI()

In [10]:
@app.post("/createDeployment/{deployment_name}")
async def create_deployment(deployment_name: str):
    try:
        # Load local kube config for local testing
        config.load_kube_config()

        k8s_apps_v1 = client.AppsV1Api()
        deployment = client.V1Deployment(
            api_version="apps/v1",
            kind="Deployment",
            metadata=client.V1ObjectMeta(name=deployment_name),
            spec=client.V1DeploymentSpec(
                replicas=1,
                selector=client.V1LabelSelector(
                    match_labels={'app': deployment_name}
                ),
                template=client.V1PodTemplateSpec(
                    metadata=client.V1ObjectMeta(labels={'app': deployment_name}),
                    spec=client.V1PodSpec(containers=[
                        client.V1Container(
                            name=deployment_name,
                            image="nginx",
                            ports=[client.V1ContainerPort(container_port=80)]
                        )
                    ])
                )
            )
        )
        k8s_apps_v1.create_namespaced_deployment(namespace="default", body=deployment)
        return {"message": f"Deployment {deployment_name} created successfully."}
    except Exception as e:
        return {"error": str(e)}

In [11]:
@app.get("/getPromdetails")
async def get_prom_details():
    try:
        # Load local kube config for local testing
        config.load_kube_config()

        k8s_api = client.CoreV1Api()
        pods = k8s_api.list_pod_for_all_namespaces().items
        pod_details = [{"name": pod.metadata.name, "namespace": pod.metadata.namespace} for pod in pods]
        return {"pod_details": pod_details}
    except Exception as e:
        return {"error": str(e)}

In [12]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, World!"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}


In [17]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, World!"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}

@app.get("/ping")
async def ping():
    return {"message": "Pong"}
